# Tailor resume for job application

Maximize your chances of getting an interview for a given job posting.

---

## Introduction

This is what the process looks like:
- Learn about the job requirements
- Cross that with your skill set and experiences
- Reshape resume to highlight relevant areas
- Rewrite resume with appropriate language
- Set talking points for your initial interview

## Setup

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
openai.api_type = os.environ.get("OPENAI_API_TYPE")
openai.api_base = os.environ.get("OPENAI_API_BASE")
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_version = os.environ.get("OPENAI_API_VERSION")

## LLM

In [ ]:
from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(
    deployment_name="gpt40125",
    temperature=0,
)

## Creating Tools

### crewAI Tools

`MDXSearchTool` allows to perform RAG over the supplied file.

In [4]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./resume/fake_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./resume/fake_resume.md')

Inserting batches in chromadb:   0%|          | 0/1 [00:00<?, ?it/s]


## Creating Agents

In [5]:
from crewai import Agent, Task, Crew

In [6]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    ),
    llm=llm,
)

In [7]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    ),
    llm=llm,
)

In [8]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    ),
    llm=llm,
)

In [9]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    ),
    llm=llm,
)

## Creating Tasks

In [10]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [11]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [12]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [13]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

In [14]:
job_application_crew = Crew(
    agents=[researcher, profiler, resume_strategist, interview_preparer],
    tasks=[
        research_task,
        profile_task,
        resume_strategy_task,
        interview_preparation_task,
    ],
    verbose=True,
)

## Running the Crew

In [15]:
job_application_inputs = {
    "job_posting_url": "https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund",
    "github_url": "https://github.com/joaomdmoura",
    "personal_writeup": """Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach.""",
}

In [16]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/joaomdmoura) URLs, and personal write-up (Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his abili

In [17]:
from IPython.display import Markdown, display

display(Markdown("./tailored_resume.md"))

# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Profile
Noah Williams is a seasoned Full Stack Engineer with over 18 years of experience in the technology industry, specializing in both front-end and back-end development. His expertise in programming languages such as Ruby, Python, JavaScript, TypeScript, and Elixir, combined with his proficiency in front-end frameworks and scalable vector databases, makes him a versatile developer capable of tackling complex full-stack projects. Noah's background in leading engineering teams and his deep understanding of AI and data science have enabled him to deliver scalable AI solutions and innovative data models, making him a valuable asset to any team.

## Technical Skills
- **Programming Languages:** Ruby, Python, JavaScript, TypeScript, Elixir
- **Front-end Technologies:** React, Angular, various front-end frameworks
- **Back-end Technologies:** Ruby on Rails, Node.js
- **Databases:** MongoDB, scalable vector databases
- **Other Skills:** AI technologies, data science, machine learning, project management

## Work History

### DataKernel: Director of Software Engineering (remote) — 2022 - Present
- Transformed the engineering division, integrating cutting-edge AI technologies and scalable vector databases, enhancing product capabilities.
- Led a substantial growth in team skill development, focusing on strategic project goals with a strong emphasis on full-stack development practices.

### DataKernel: Senior Software Engineering Manager (remote) — 2019 - 2022
- Directed engineering strategy and operations, fostering a culture of continuous improvement and high performance in full-stack development.

### InnovPet: Founder & CEO (remote) — 2019 - 2022
- Founded InnovPet, overseeing product development from concept through execution, emphasizing full-stack development for IoT solutions.

### EliteDevs: Engineering Manager (remote) — 2018 - 2019
- Formulated strategic plans enhancing project outcomes, managed engineering teams with a focus on full-stack development using Ruby on Rails and React.

### PrintPack: Engineering Manager (remote) — 2016 - 2018
- Led the development of a predictive modeling tool, showcasing proficiency in back-end technologies and data analytics.

### DriveAI: Senior Software Engineer (remote) — 2015 - 2016
- Developed and optimized a central API, demonstrating strong back-end development skills and advanced caching strategies.

### BetCraft: CTO — 2013 - 2015
- Led technology strategy, improving platform performance with a focus on full-stack development using React and Angular.

## Education

### MBA in Information Technology
London Business School - MBA

### Advanced Leadership Techniques
University of London - Certification

### Data Science Specialization
Coursera (Johns Hopkins University) - Certification

### B.Sc. in Computer Science
University of Edinburgh - Bachelor’s degree

Noah Williams is an ideal candidate for the Full Stack Engineer position at AI Fund, offering a robust blend of technical expertise in both front-end and back-end development, leadership, and strategic innovation.

In [19]:
display(Markdown("./interview_materials.md"))

**Interview Questions and Talking Points for Noah Williams for the Full Stack Engineer Position at AI Fund**

1. **Technical Expertise and Project Experience**
   - Can you discuss your role in the development of crewAI and how it demonstrates your ability to produce scalable software solutions?
   - How did your work on machinery and active_model_serializers contribute to the full software development life cycle, from conception to deployment?
   - Given your experience with both front-end and back-end technologies, how do you approach choosing the right stack for a project?

2. **Leadership and Team Collaboration**
   - You've led remote teams in various capacities. How do you ensure effective communication and collaboration among cross-functional teams in a remote setting?
   - Can you provide an example of a challenging project at DataKernel or InnovPet and how you led your team to success?

3. **Innovation and Problem-Solving**
   - AI Fund values innovative solutions. Can you talk about a time when you had to think outside the box to solve a software development problem?
   - How do you stay updated with the latest technologies and frameworks, and how have you applied this knowledge in your projects?

4. **Technical Skills Match**
   - You have experience with a wide range of programming languages and frameworks. How do you think your skills with Python, Ruby, JavaScript, React, and Angular will contribute to our projects at AI Fund?
   - Given your background in AI technologies and data science, how do you see yourself contributing to AI Fund's focus on producing scalable software solutions for portfolio companies?

5. **Cultural Fit and Personal Attributes**
   - AI Fund values accountability, grit, scrappiness, and ownership orientation. Can you share an experience that demonstrates these characteristics?
   - How do you align with AI Fund's commitment to diversity and inclusion in the workplace?

6. **Future Contributions and Growth**
   - Looking at AI Fund's current projects and future goals, where do you see yourself making the most significant impact?
   - How do you plan to continue growing professionally and technically in the role of a Full Stack Engineer at AI Fund?

**Conclusion:**
These questions and talking points are designed to help Noah Williams effectively showcase his extensive experience, technical skills, leadership qualities, and innovative approach to software development. They align with the job requirements and responsibilities of the Full Stack Engineer position at AI Fund, providing a comprehensive framework for the interview process.